### Prapare database connections

In [1]:
import logica_driver as driver
from sqlalchemy import create_engine
engine = create_engine("sqlite:////db/movies.db")
connection = engine.connect()
driver.SetDbConnection(connection)

### Basic logic deduction (not working as sqlite doesn't support UNNEST operation)

In [3]:
%%logica Prime
@Engine("sqlite");

# Define natural numbers from 1 to 29.
N(x) :- x in Range(30);
# Define primes.
Prime(prime: x) :-
  N(x),
  x > 1,
  ~(
    N(y),
    y > 1,
    y != x,
    Mod(x, y) == 0
  );

Running Prime
The following query is stored at Prime_sql variable.
SELECT
  x_4 AS prime
FROM
  UNNEST(GENERATE_ARRAY(0, 30 - 1)) as x_4
WHERE
  (x_4 > 1) AND
  ((SELECT
    MIN(1) AS logica_value
  FROM
    UNNEST(GENERATE_ARRAY(0, 30 - 1)) as x_7
  WHERE
    (x_7 > 1) AND
    (x_7 != x_4) AND
    MOD(x_4, x_7) = 0) IS NULL);


OperationalError: (sqlite3.OperationalError) no such table: UNNEST
[SQL: SELECT
  x_4 AS prime
FROM
  UNNEST(GENERATE_ARRAY(0, 30 - 1)) as x_4
WHERE
  (x_4 > 1) AND
  ((SELECT
    MIN(1) AS logica_value
  FROM
    UNNEST(GENERATE_ARRAY(0, 30 - 1)) as x_7
  WHERE
    (x_7 > 1) AND
    (x_7 != x_4) AND
    MOD(x_4, x_7) = 0) IS NULL)]
(Background on this error at: http://sqlalche.me/e/14/e3q8)

### Database Demos

In [15]:
%%logica PeopleId
@Engine("sqlite");

# Find people with given id
PeopleId(id:, name:) :-
    people(id: id, name: name),
    id == 11012483


Running PeopleId
The following query is stored at PeopleId_sql variable.
SELECT
  people.id AS id,
  people.name AS name
FROM
  people
WHERE
  people.id = 11012483;
The following table is stored at PeopleId variable.


,id,name
0,11012483,Balkar Singh Bali


In [18]:
%%logica AandD
@Engine("sqlite");

# people who is both an actor and a director and born after 1990
AandD(name:) distinct :-
    people(id: id, name: name, birth: birth),
    stars(person_id: id),
    directors(person_id: id),
    birth > 1990


Running AandD
The following query is stored at AandD_sql variable.
SELECT
  people.name AS name
FROM
  people, stars, directors
WHERE
  (people.birth > 1990) AND
  stars.person_id = people.id AND
  directors.person_id = people.id
GROUP BY name;
The following table is stored at AandD variable.


,name
0,Adam Starks
1,Akin Sipal
2,Al Gurst
3,Alex R. Wagner
4,Alex Tamaro
...,...
91,Tomas Simonsen
92,Yakov Kolontarov
93,Yana Gladkikh
94,Zuher Khan


In [23]:
%%logica AandDMovies
@Engine("sqlite");

# Movies whose directors appears in them
AandDMovies(name:, title:) :-
    people(id: id, name: name),
    stars(person_id: id, movie_id: mid),
    directors(person_id: id, movie_id: mid),
    movies(id: mid, title: title)

Running AandDMovies
The following query is stored at AandDMovies_sql variable.
SELECT
  people.name AS name,
  movies.title AS title
FROM
  people, stars, directors, movies
WHERE
  stars.person_id = people.id AND
  directors.person_id = people.id AND
  directors.movie_id = stars.movie_id AND
  movies.id = stars.movie_id;
The following table is stored at AandDMovies variable.


,name,title
